In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# **MNIST Keras **
### ** Cong Vo, Student**

* **1. Introduction**

* **2. Data Preparation**
    * 2.1. Load data
    * 2.2. Check for null and missing values
    * 2.3. Normalization
    * 2.4. Reshape
    * 2.5. Label Encoding
    * 2.6. Split training and validation set
    
* **3. CNN**
    * 3.1. Defind the model
    * 3.2. Set the optimizer and annealer



# 1. Introduction
This notebook is to practice how to build a Deep Learning model on MNIST dataset
We built a 5 layers Sequential Convolution Neural Network using Tensorflow instead of Keras.
This will be more challenged to me.

# 2. Data Preparation

### 2.1. Loading data

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
training_dataset = pd.read_csv("../input/train.csv")
testing_dataset = pd.read_csv("../input/test.csv")

In [ ]:
training_dataset.head(5)

In [ ]:
training_dataset['label'].unique()

In [ ]:
training_dataset['label'].count()

In [ ]:
Y_train = training_dataset['label']

In [ ]:
X_train = training_dataset.drop(labels=['label'], axis=1)

In [ ]:
X_test = testing_dataset.values

In [ ]:
X_test.shape

### 2.2. Check for null and missing data

In [ ]:
X_train.isnull().any().describe()

In [ ]:
Y_train.isnull().any()

### 2.3. Normalization

In [ ]:
X_train = X_train/255.0

In [ ]:
X_test = X_test/ 255.0

In [ ]:
X_train.shape

In [ ]:
X_train.values

In [ ]:
X_test.shape

### 2.3. Reshape

In [ ]:
# Reshape image in 3 dimensions (height = 28px, width = 28px, canal = 1)
X_train = X_train.values.reshape(-1, 28, 28, 1)

In [ ]:
X_test = X_test.reshape([-1, 28, 28, 1])

### 2.5. Label Encoding

In [ ]:
Y_train = to_categorical(Y_train, num_classes=10)

### 2.6. Split training and validation set

In [ ]:
random_seed = 2

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, random_state=random_seed, test_size = 0.3)

In this part, I splited training data into 2 subparts: training part and validation part.

### 3.CNN

In this part I'm going to use Keras to build model graph.
The first is convolutional layer. I chose 32 filter size for the two first layer and 64 filter size for the two last ones. 
The second important layer is the pooling layer (Maxpooling). This layer simply acts as downsampling filter. It look at 4 neighbor pixels and keep the maximal one. That can help reducing computational cost, and to some extent also reduce overfitting. Pooling size in this layer is 2x2

Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored. This technique can help improve generalization and reduces the overfitting.

ReLU is the rectifier. This term is to add non-linearity to the network


In [ ]:
# Model Architecture
# Input -> [Conv2d (5, 5) -> Maxpooling (2, 2) -> Dropout] x2 -> [Conv2d (3, 3) -> Maxpooling (2, 2) -> Dropout] x2 -> Flatten -> Classify

In [ ]:
from keras import Sequential
from keras.layers import Dense, MaxPooling2D, Dropout, Flatten, Conv2D

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(padding='same', pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPooling2D(padding='same', pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(padding='same', pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(padding='same', pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'] )


In [ ]:
model.fit(batch_size=200, epochs=12, x=X_train, y=Y_train, validation_data=[X_val, Y_val])

In [ ]:
prediction = model.predict(X_test)

In [ ]:
label = np.argmax(prediction, axis=1)

In [ ]:
test_id = np.reshape(range(1, len(prediction) + 1), label.shape)

In [ ]:
my_submission = pd.DataFrame({'ImageId': test_id, 'Label': label})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)